# Imports

In [1]:
import zipfile
import json
from scipy import sparse
import numpy as np
import time
import scipy.sparse.linalg as lg
import scipy.cluster.vq as vq

# Dataset path

In [2]:
datadirectory="/home/leboudy/Downloads/lastfm_test.zip"
dataset = zipfile.ZipFile(datadirectory)

# global variables initialization


In [3]:
songId=0
t=0
k=10
#g=50
#beta=0.2
#categories =["Technical Death Metal"]
#S =0#[]
#id2songJsonMap = []
id2songIdMap = {}
#fhat=[]
#n=5 #top songs to report
#limitingCounter = 1000
kmeansIterations=100

# Loading data


In [4]:
start_time = time.time()
for i in dataset.namelist():
    if  i.endswith(".json"): 
        f = json.loads(dataset.read(i).decode("utf-8"))
        if  f["track_id"] not in id2songIdMap.keys():
            id2songIdMap[f["track_id"]] = (songId,i)
            #tempcategories = set(categories)#will keep removing tags from it
            #newtags=[]
            #for tag in f["tags"]:#looping over tags
             #   if int(tag[1]) >g:#filters out those <g
              #      newtags.append(tag)#keeping >g
               #     if tag[0] in tempcategories:#if it is in the tempcategories
                #        tempcategories.remove(tag[0])#removing it
            #f["tags"] = newtags
            #id2songJsonMap.append(f)
            #songTags= [t[0] for t in f["tags"]]
            #if len(list(filter(lambda x: x in categories,songTags )) ) == len(categories):
            #if len(tempcategories)==0:#meaning we encountered all of the tags
             #   S+=1#.append(songId)
              #  fhat.append(1)
            #else:
             #   fhat.append(0)
            songId+=1 
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.300939798355103 seconds ---


# initializing the teleportation probablilities


In [5]:
#_fhat=np.array()
#_fhat.shape=(songId,1)
#fhat=(1/len(S))*np.array(fhat)
#fhat=(1/S)*np.array(fhat)

# Building the Adjacency matrix


In [6]:
start_time = time.time()
col=[]
row=[]
data=[]
for i in id2songIdMap.keys():
    f = json.loads(dataset.read(id2songIdMap[i][1]).decode("utf-8"))
    for s in f["similars"]:
        if s[0] in id2songIdMap.keys() and s[1]>=t:
            #similar = json.loads(dataset.read(id2songIdMap[s[0]][1]).decode("utf-8"))
            val=s[1]
            #if i in similar["similars"].keys():
            #    val =max(val,similar["similars"][i])

                
            col.append(id2songIdMap[i][0])
            row.append(id2songIdMap[s[0]][0])
            data.append(val)
            
            #row.append(id2songIdMap[i][0])
            #col.append(id2songIdMap[s[0]][0])
            #data.append(val)
    
    
    

        
print("--- %s seconds ---" % (time.time() - start_time))        


--- 25.218172550201416 seconds ---


In [7]:
A = sparse.csc_matrix((data, (row, col)),shape=(songId, songId))


In [12]:
print(A.shape)

(104212, 104212)


In [13]:
def maximum(A,B):
    BisBigger = A-B
    BisBigger.data = np.where(BisBigger.data<0,1,0)
    return A-A.multiply(BisBigger)+B.multiply(BisBigger)

In [14]:
W = maximum(A,A.T)


In [15]:
W.shape

(104212, 104212)

In [16]:
len(list(W.sum(0).flat))
D = sparse.csc_matrix((list(W.sum(0).flat), (range(songId), range(songId))))

In [17]:
np.sum(W.sum(axis=0)[0])

278922.6826198668

In [18]:
L=D-W

In [19]:
L

<104212x104212 sparse matrix of type '<class 'numpy.float64'>'
	with 1704095 stored elements in Compressed Sparse Column format>

# Computing the Smallest K eignvectors

In [20]:
#w,v = lg.eigs(L,k=k,sigma=0)

In [21]:
#s,v,d = lg.svds(L,k=5,which='SM')
_,largestEigen,_= lg.svds(L,k=1)

In [22]:
print (largestEigen)

[ 4002.66891005]


In [23]:
I = sparse.identity(songId, format='csc')
I.setdiag(largestEigen[0])

In [24]:
I[2,2]

4002.6689100461372

In [25]:
B = L-I

In [35]:
e,v= lg.eigs(B,k=k,return_eigenvectors=True, which='LM')

KeyboardInterrupt: 

# Randomly initializing r


In [29]:
e+=largestEigen

In [32]:
e

array([  3.94831722e-03+0.j,   2.92297324e-01+0.j,   8.50698578e-01+0.j,
         1.67218405e+00+0.j,   2.69640668e+00+0.j,   3.93036808e+00+0.j,
         5.40366236e+00+0.j,   7.09468729e+00+0.j,   9.06742414e+00+0.j,
         1.13409873e+01+0.j])

In [30]:
#e,v= lg.eigs(B,k=k,return_eigenvectors=True, sigma=-13)

In [31]:
#e
print (songId)

104212


In [ ]:
#e,v,s= lg.svds(B,k=k,tol=1e-3)
#e

In [ ]:
#v

In [33]:

c,l=vq.kmeans2(v.real,k,kmeansIterations)

In [34]:
l

array([5, 2, 6, ..., 9, 7, 7], dtype=int32)

In [ ]:
r =np.random.rand(A.shape[0])
r=r/np.sum(r)

In [ ]:
#print(len(id2songJsonMap))
print (A.shape)
print(songId)
print (S)
print (songId)

# Power Iterations


In [ ]:
for i in range(100):
    r=A.multiply(beta).dot(r)+fhat*(1-beta)

# Reporting the top n songs


In [ ]:
topIndecs = r.argsort()[-n:][::-1]


In [ ]:
keys= list(id2songIdMap.keys())
for item in topIndecs:
    print (keys[item])

In [ ]:
print (topIndecs)

In [ ]:
A =np.asarray( [[2,3],[1,6]])

In [ ]:
A

In [ ]:
np.maximum(A, A.T)